### About this demo

This demo test the time cost between different PennyLane quantum devices.

**Additional packages required:**
- pip install "pennylane-qulacs[cpu]"
- pip install pennylane-lightning

In [1]:
import os
import sys
import time

import pennylane as qml
import torch
import torch.nn as nn

sys.path.append(os.path.abspath('../'))

from source.models.qcgnn import QuantumRotQCGNN

os.environ['OMP_NUM_THREADS'] = '8'

qml.about()

Name: PennyLane
Version: 0.36.0
Summary: PennyLane is a cross-platform Python library for quantum computing, quantum machine learning, and quantum chemistry. Train a quantum computer the same way as a neural network.
Home-page: https://github.com/PennyLaneAI/pennylane
Author: 
Author-email: 
License: Apache License 2.0
Location: /home/yianchen/.pyenv/versions/3.12.2/lib/python3.12/site-packages
Requires: appdirs, autograd, autoray, cachetools, networkx, numpy, pennylane-lightning, requests, rustworkx, scipy, semantic-version, toml, typing-extensions
Required-by: PennyLane-qiskit, pennylane-qulacs, PennyLane_Lightning, QCGNN

Platform info:           Linux-5.14.0-362.8.1.el9_3.x86_64-x86_64-with-glibc2.34
Python version:          3.12.2
Numpy version:           1.26.4
Scipy version:           1.13.0
Installed devices:
- lightning.qubit (PennyLane_Lightning-0.36.0)
- default.clifford (PennyLane-0.36.0)
- default.gaussian (PennyLane-0.36.0)
- default.mixed (PennyLane-0.36.0)
- default.qub

In [2]:
q_setup = [
    ('default.qubit', 'best'),
    ('lightning.qubit', 'adjoint'),
    ('qulacs.simulator', 'best'),
]

batch_size = 128
num_ptcs = 16
score_dim = 3

x = torch.rand(batch_size, num_ptcs, 3)
y = torch.randint(low=0, high=score_dim, size=(batch_size, ))

for dev, diff_method in q_setup:

    time_start = time.time()
    
    model = QuantumRotQCGNN(
        num_ir_qubits=4,
        num_nr_qubits=3,
        num_layers=1,
        num_reupload=1,
        score_dim=score_dim,
        qdevice=dev,
        qbackend='qiskit',
        diff_method=diff_method,
    )
    
    optimizer = torch.optim.RAdam(model.parameters(), lr=1E-2)
    loss_func = nn.CrossEntropyLoss()

    for epoch in range(5):
        optimizer.zero_grad()
        loss = loss_func(model(x), y)
        loss.backward()
        optimizer.step()

        print(f"- Epoch: {epoch} | Loss: {loss.item():.4f}")
    
    time_end = time.time()

    print(f"* Device: {dev} | Time: {time_end - time_start:.2f} s\n")

# ModelLog: Quantum device  = default.qubit
# ModelLog: Quantum backend = qiskit
# ModelLog: Qubits (IR, WK, NR) = ((4, 3, 3))
- Epoch: 0 | Loss: 12.7624
- Epoch: 1 | Loss: 23.1522
- Epoch: 2 | Loss: 19.9279
- Epoch: 3 | Loss: 12.8589
- Epoch: 4 | Loss: 6.7814
* Device: default.qubit | Time: 2.84 s

# ModelLog: Quantum device  = lightning.qubit
# ModelLog: Quantum backend = qiskit
# ModelLog: Qubits (IR, WK, NR) = ((4, 3, 3))
- Epoch: 0 | Loss: 8.1098
- Epoch: 1 | Loss: 15.5815
- Epoch: 2 | Loss: 13.4436
- Epoch: 3 | Loss: 4.4681
- Epoch: 4 | Loss: 1.7048
* Device: lightning.qubit | Time: 29.39 s

# ModelLog: Quantum device  = qulacs.simulator
# ModelLog: Quantum backend = qiskit
# ModelLog: Qubits (IR, WK, NR) = ((4, 3, 3))
